In [27]:
import json
import random

def carregar_perguntas_por_arquivo(categoria, nivel=None):
    caminho = f"perguntas/{categoria.lower()}.json"
    try:
        with open(caminho, 'r', encoding='utf-8') as f:
            perguntas = json.load(f)
        if nivel:
            perguntas = [p for p in perguntas if p['nivel'].lower() == nivel.lower()]
        random.shuffle(perguntas)
        return perguntas
    except FileNotFoundError:
        print(f"⚠️ Arquivo não encontrado para categoria '{categoria}'.")
        return []

In [28]:
import time
import csv
import os

def rodar_quiz(perguntas, jogador, tempo_limite=15):
    pontuacao = 0
    respostas_certas = []
    respostas_erradas = []

    for i, q in enumerate(perguntas):
        print(f"\nPergunta {i+1}: {q['pergunta']}")
        for idx, opcao in enumerate(q['opcoes']):
            print(f"{idx}. {opcao}")

        inicio = time.time()
        resposta = None

        while True:
            try:
                resposta = int(input(f"Digite o número da resposta (tempo limite: {tempo_limite}s): "))
                if resposta < 0 or resposta >= len(q['opcoes']):
                    print("⚠️ Opção inválida. Tente novamente.")
                    continue
                break
            except ValueError:
                print("⚠️ Entrada inválida. Digite um número.")
            if time.time() - inicio > tempo_limite:
                print("⏱️ Tempo esgotado!")
                resposta = None
                break

        if resposta == q['resposta']:
            print("✅ Correto!", {q['extra']})
            pontuacao += 1
            respostas_certas.append(q['pergunta'])
        else:
            print(f"❌ Errado! A resposta correta era: {q['opcoes'][q['resposta']]}")
            respostas_erradas.append(q['pergunta'])

    print(f"\nPontuação final de {jogador}: {pontuacao}/{len(perguntas)}")
    salvar_resultado_csv(jogador, pontuacao, len(perguntas))
    return pontuacao


In [29]:
def salvar_resultado_csv(jogador, pontuacao, total):
    arquivo = 'ranking.csv'
    existe = os.path.isfile(arquivo)
    with open(arquivo, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        if not existe:
            writer.writerow(['Jogador', 'Pontuação', 'Total'])
        writer.writerow([jogador, pontuacao, total])

In [30]:
def mostrar_ranking():
    try:
        with open('ranking.csv', 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)  # pular cabeçalho
            ranking = sorted(reader, key=lambda x: int(x[1]), reverse=True)
            print("\n🏆 Ranking:")
            for i, linha in enumerate(ranking[:5]):
                print(f"{i+1}. {linha[0]} - {linha[1]}/{linha[2]}")
    except FileNotFoundError:
        print("Nenhum ranking disponível ainda.")

In [31]:
jogador = input("Digite seu nome: ")
categoria = input("Escolha uma categoria (Programming, Web, DevOps): ")
nivel = input("Escolha o nível (Fácil, Médio, Difícil): ")

perguntas = carregar_perguntas_por_arquivo(categoria, nivel)

if not perguntas:
    print("⚠️ Nenhuma pergunta encontrada.")
else:
    pontuacao = rodar_quiz(perguntas, jogador)
    mostrar_ranking()

Digite seu nome:  Ale
Escolha uma categoria (Programming, Web, DevOps):  Web
Escolha o nível (Fácil, Médio, Difícil):  Fácil



Pergunta 1: O que significa a sigla URL?
0. Universal Resource Locator 
1. Uniform Resource Locator
2. Unique Reference Link
3. User Remote Login


Digite o número da resposta (tempo limite: 15s):  1


✅ Correto! {'Explicação: URL é o endereço de um recurso na web, como uma página ou arquivo.'}

Pergunta 2: Qual é a função principal do protocolo HTTP?
0. Criptografar dados entre cliente e servidor
1. Transferir arquivos entre computadores
2. Exibir páginas web no navegador
3. Controlar o acesso à rede local


Digite o número da resposta (tempo limite: 15s):  2


✅ Correto! {'Explicação: O HTTP (HyperText Transfer Protocol) é usado para transmitir páginas web entre servidores e navegadores.'}

Pontuação final de Ale: 2/2

🏆 Ranking:
1. Ale - 2/2
2. Ale - 1/1
3. Ale - 1/1
4. Ale - 0/2
